In [2]:
import sys
!{sys.executable} -m pip install zmanim

  Created wheel for julian: filename=julian-0.14-py3-none-any.whl size=2632 sha256=d3d11bca8a415e973d6f8428b62efad067afcd6a3a7e22a9ac963ae321c5e96f
  Stored in directory: c:\users\nathan\appdata\local\pip\cache\wheels\ac\43\c7\fc558ec7c502b57cf2e925b95b6686ff5553da0c60339cf8b3
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=44702 sha256=55c8e2ca058aecd6eb081b1c2e55fd832374e3272bc603e1aa80b8300aefb7cd
  Stored in directory: c:\users\nathan\appdata\local\pip\cache\wheels\a7\cb\f2\25802a0599b033b24cbf2cb0210af6f13c27291db92cd5178c
Successfully built julian memoization


In [1]:
from zmanim.zmanim_calendar import ZmanimCalendar
from zmanim.astronomical_calendar import AstronomicalCalendar
from zmanim.hebrew_calendar.jewish_calendar import JewishCalendar
from zmanim.util.geo_location import GeoLocation
from zmanim.hebrew_calendar.jewish_date import JewishDate
import urllib.request, json, datetime, time, pytz
import datetime

In [2]:
def misheyakircalc(self, opts: dict = {'degrees': 11}):
    degrees, offset, zmanis_offset = self._extract_degrees_offset(opts)
    sunrise_for_degrees = self.elevation_adjusted_sunrise() if degrees == 0 else self.sunrise_offset_by_degrees(self.GEOMETRIC_ZENITH + degrees)
    if zmanis_offset != 0:
        return self._offset_by_minutes_zmanis(sunrise_for_degrees, -zmanis_offset)
    else:
        return self._offset_by_minutes(sunrise_for_degrees, -offset)

In [3]:
def duskcalc(self, opts: dict = {'degrees': 6.45}):
    degrees, offset, zmanis_offset = self._extract_degrees_offset(opts)
    sunset_for_degrees = self.elevation_adjusted_sunset() if degrees == 0 else self.sunset_offset_by_degrees(self.GEOMETRIC_ZENITH + degrees)
    if zmanis_offset != 0:
        return self._offset_by_minutes_zmanis(sunset_for_degrees, zmanis_offset)
    else:
        return self._offset_by_minutes(sunset_for_degrees, offset)

In [4]:
def zmanimcalc(now_local, zmanloc):
    calendar = ZmanimCalendar(geo_location=zmanloc, date=now_local)
    
    zmanlist = []
    zmanlist.append(calendar.alos())
    zmanlist.append(misheyakircalc(calendar))
    zmanlist.append(calendar.sunrise())
    zmanlist.append(calendar.sof_zman_shma_gra())
    zmanlist.append(calendar.sof_zman_tfila_gra())
    zmanlist.append(calendar.chatzos())
    #zmanlist.append(calendar.mincha_gedola())
    zmanlist.append(calendar.mincha_ketana())
    zmanlist.append(calendar.plag_hamincha())
    zmanlist.append(calendar.sunset())
    zmanlist.append(duskcalc(calendar))
    zmanlist.append(calendar.tzais())
    
    hour = (calendar.sunset() - calendar.sunrise())/12
    
    output = now_local.strftime("%m-%d")
    for zman in zmanlist:
        output += "&"+dt_to_time(zman)
    output += "&" + str(round(hour.total_seconds()/60))
    output += r"\\\hline"
    return output

In [5]:
def dt_to_time(dtobj):
    if dtobj.dst() != datetime.timedelta(seconds=0):
        dtobj = dtobj - dtobj.dst()
    return dtobj.strftime("%H:%M")

In [6]:
def run_year(location):
    start_date = datetime.datetime(2023,1,1)
    end_date = datetime.datetime(2024,1,1)
    eachdate = start_date
    lines = []
    while eachdate<end_date:
        line = zmanimcalc(eachdate,location)
        lines.append(line)
        eachdate = eachdate + datetime.timedelta(days=3)
    return lines

In [9]:
def add_template(lines, title):
    output = [r"\section{Zmanim for "+title+" }"]
    output.append("")
    output.append(r"\begin{footnotesize}")
    output.append(r"\begin{longtable}{c | c | c | c | c | c | c | c | c | c | c | c | c}")
    output.append(r"Date&Dawn&Misheyakir&Sunrise&Shema&Sha\d{h}arit&Noon&MK&Plag&Sunset&Dusk&Nightfall&Prop Hr\\\hline\endhead")
    for line in lines:
        output.append(line)
    output.append(r"\end{longtable}\end{footnotesize}")
    return output

In [10]:
location = GeoLocation('Chicago, IL', 41.942, -87.644, 'America/Chicago', elevation=0)
lines = run_year(location)
text = add_template(lines,"Chicago")
for line in text:
    print(line)

\section{Zmanim for Chicago }

\begin{footnotesize}
\begin{longtable}{c | c | c | c | c | c | c | c | c | c | c | c | c}
Date&Dawn&Misheyakir&Sunrise&Shema&Sha\d{h}arit&Noon&MK&Plag&Sunset&Dusk&Nightfall&Prop Hr\\\hline\endhead
01-01&05:48&06:17&07:18&09:36&10:22&11:54&14:35&15:32&16:29&17:04&17:16&46\\\hline
01-04&05:48&06:17&07:18&09:37&10:23&11:55&14:37&15:34&16:32&17:06&17:18&46\\\hline
01-07&05:49&06:17&07:18&09:37&10:24&11:56&14:39&15:37&16:35&17:09&17:21&46\\\hline
01-10&05:48&06:17&07:17&09:37&10:24&11:58&14:41&15:40&16:38&17:12&17:24&47\\\hline
01-13&05:48&06:17&07:16&09:38&10:25&11:59&14:44&15:43&16:41&17:15&17:27&47\\\hline
01-16&05:47&06:16&07:15&09:38&10:25&12:00&14:46&15:46&16:45&17:18&17:30&47\\\hline
01-19&05:46&06:14&07:13&09:37&10:25&12:01&14:49&15:49&16:49&17:22&17:33&48\\\hline
01-22&05:44&06:13&07:11&09:37&10:25&12:02&14:51&15:52&16:52&17:25&17:37&48\\\hline
01-25&05:43&06:11&07:09&09:36&10:25&12:03&14:54&15:55&16:56&17:29&17:40&49\\\hline
01-28&05:41&06:09&07:07&0